# RNA STAR
Here we perform sequence alignments on the input FASTQ files. This produces `.sam` files.

In [ ]:
!apt install rna-star

# sambamba
Here we convert the `.sam` files to `.bam` files and sort them.

In [8]:
!# password.txt is a text file with your password.
!# If this gives an error (probably because you don't have sudo permissions),
!# try running apt install -y sambamba instead.
!sudo -S apt install -y sambamba < password.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libllvm11 libphobos2-ldc-shared98
The following NEW packages will be installed:
  libllvm11 libphobos2-ldc-shared98 sambamba
0 upgraded, 3 newly installed, 0 to remove and 1 not upgraded.
Need to get 21.8 MB of archives.
After this operation, 92.9 MB of additional disk space will be used.
Get:1 http://mirrors.ocf.berkeley.edu/ubuntu jammy/universe amd64 libllvm11 amd64 1:11.1.0-6 [19.6 MB]
Get:2 http://mirrors.ocf.berkeley.edu/ubuntu jammy/universe amd64 libphobos2-ldc-shared98 amd64 1:1.28.0-1ubuntu1 [1,487 kB]
Get:3 http://mirrors.ocf.berkeley.edu/ubuntu jammy/universe amd64 sambamba amd64 0.8.2+dfsg-2 [695 kB]
Fetched 21.8 MB in 1s (21.4 MB/s)m

78Selecting previously unselected package libllvm11:amd64.
(Reading database ... 414490 files and directories currently installed.)Reading database ... 65%(Reading database ... 70%(Reading 

In [12]:
%%bash

# note that bash scripts don't make output in Jupyter notebooks until they finish (sad!)

mkdir -p ./sorted_bam
for dir in results/*/; do
  echo dir $dir
  if [ -d "$dir" ]; then
    for subdir in "$dir"/*/; do
      subdir_name=$(basename "$subdir")
      if [ -f "$subdir/Aligned.out.sam" ]; then
        sam_file="$subdir/Aligned.out.sam"
        group="${dir:9:3}"
        short="${subdir_name:0:12}"
        output=$group-$short
        sambamba view -S --format=bam $sam_file | sambamba sort -n -o sorted_bam/${output}.bam /dev/stdin
      fi
    done
  fi
done


dir results/00MatrigelCoating/
dir results/00PLLCoating/
dir results/01agrin/
dir results/02PanLaminin/
dir results/03Laminin211/
dir results/04CollagenI/
dir results/_STARtmp/




sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)

    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021

    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)

    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)

    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021sambam

In [6]:
import pathlib
import os
import subprocess

pathlib.Path("sorted_bam").mkdir(exist_ok=True)
for folder in os.listdir("results"):
    folderpath = f"results/{folder}"
    if not os.path.isdir(folderpath):
        continue
    
    for subfolder in os.listdir(folderpath):
        subfolderpath = f"{folderpath}/{subfolder}"
        sam_file = f"{subfolderpath}/Aligned.out.sam"
        if os.path.isfile(sam_file):
            group = folder[9:12]
            short = subfolder[0:13]
            output = f"{group}-{short}"
            # run sambamba on the file and put it in sorted_bam
            subprocess.run(f"sambamba view -S --format=bam {sam_file} | sambamba sort -n -o sorted_bam/{output}.bam /dev/stdin", shell=True, check=True)
            print(f"Ran sambamba on {subfolderpath}/{sam_file}")


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



Ran sambamba on results/01agrin/lib103916_L2-ds.d1b626e6358749c5aa8c7a900453bf55/results/01agrin/lib103916_L2-ds.d1b626e6358749c5aa8c7a900453bf55/Aligned.out.sam



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



Ran sambamba on results/01agrin/lib103917_L1-ds.6cb7a56486a743aa97aef4b4492c71db/results/01agrin/lib103917_L1-ds.6cb7a56486a743aa97aef4b4492c71db/Aligned.out.sam



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021

sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)

    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



Ran sambamba on results/01agrin/lib103916_L1-ds.cf006f61b73c426b81c8530ea73ed130/results/01agrin/lib103916_L1-ds.cf006f61b73c426b81c8530ea73ed130/Aligned.out.sam



sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.28.0 / DMD v2.098.0 / LLVM13.0.0 / bootstrap LDC - the LLVM D compiler (1.28.0)



KeyboardInterrupt: 

# htseq-count
Here we get sequence counts, and store them in CSV files.

In [7]:
%%bash
prefixes=("0M" "0P" "1a" "2P" "3L" "4C")
for pre in "${prefixes[@]}"; do
  bam_files=()
  for bam in sorted_bam/"$pre"*.bam; do
    bam_files+=($bam)
  done
  echo counting ${bam_files[@]}
    htseq-count --format=bam --order=name --stranded=yes "${bam_files[@]}" Homo_sapiens.GRCh38.113.gtf > htseqModResults/"$pre"_output.csv
  echo output to htseqModResults/"$pre"_output.csv
done


sorting sorted_bam/1ag-lib103915_L1.bam ...
sorting sorted_bam/1ag-lib103915_L2.bam ...
sorting sorted_bam/1ag-lib103916_L1.bam ...
sorting sorted_bam/1ag-lib103916_L2.bam ...
sorting sorted_bam/1ag-lib103917_L1.bam ...
sorting sorted_bam/1ag-lib103917_L2.bam ...
output to htseqResults/1a_output.csv


100000 GFF lines processed.
200000 GFF lines processed.
300000 GFF lines processed.
400000 GFF lines processed.
500000 GFF lines processed.
600000 GFF lines processed.
700000 GFF lines processed.
800000 GFF lines processed.
900000 GFF lines processed.
1000000 GFF lines processed.
1100000 GFF lines processed.
1200000 GFF lines processed.
1300000 GFF lines processed.
1400000 GFF lines processed.
1500000 GFF lines processed.
1600000 GFF lines processed.
1700000 GFF lines processed.
1800000 GFF lines processed.
1900000 GFF lines processed.
2000000 GFF lines processed.
2100000 GFF lines processed.
2200000 GFF lines processed.
2300000 GFF lines processed.
2400000 GFF lines processed.
2500000 GFF lines processed.
2600000 GFF lines processed.
2700000 GFF lines processed.
2800000 GFF lines processed.
2900000 GFF lines processed.
3000000 GFF lines processed.
3100000 GFF lines processed.
3200000 GFF lines processed.
3300000 GFF lines processed.
3400000 GFF lines processed.
3500000 GFF lines proce

# DESeq 2
This is used to get differentially expressed genes.

In [6]:
!Rscript deseq2.R

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org
Bioconductor version 3.14 (BiocManager 1.30.25), R 4.1.2 (2021-11-01)
Installing package(s) 'DESeq2'
also installing the dependencies ‘Biostrings’, ‘curl’, ‘RCurl’, ‘KEGGREST’, ‘httr’, ‘GenomeInfoDb’, ‘AnnotationDbi’, ‘annotate’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘genefilter’, ‘geneplotter’

trying URL 'https://bioconductor.org/packages/3.14/bioc/src/contrib/Biostrings_2.62.0.tar.gz'
Content type 'application/octet-stream' length 12645827 bytes (12.1 MB)
downloaded 12.1 MB

trying URL 'https://cloud.r-project.org/src/contrib/curl_6.0.1.tar.gz'
Content type 'application/x-gzip' length 932716 bytes (910 KB)
downloaded 910 KB

trying URL 'https://cloud.r-project.org/src/contrib/RCurl_1.98-1.16.tar.gz'
Content type 'application/x-gzip' length 728943 bytes (711 KB)
downloaded 711 KB

trying U

# ChEA3
This script uses the ChEA3 API to perform transcription factor network analysis.

In [24]:
# Run ChEA3 on each dataframe

import csv
import requests
import pandas as pd
import sanbomics.tools

MATRIGEL = "00Matrigel"
PLL = "00PLL"
AGRIN = "01Agrin"
PAN_LAMANIN = "02Pan-lamanin"
LAMANIN211 = "03Lamanin211"
COLLAGENI = "04Collagen_I"

# load the gene names from a CSV file.
def get_gene_set(filename) -> list:
    with open(filename, "r") as f:
        data = csv.reader(f)

        # return item in first column of each row (gene name)
        return list(line[0] for line in data)

# converts names like ENSG00000188157 to AGRN
def map_genes(ensg_names: list) -> list:
    mapper = sanbomics.tools.id_map("human").mapper
    return list(mapper[gene_name] for gene_name in ensg_names)

# Creates a CSV file with ChEA3 using the given differentailly expressed
# gene set and output filename.
def make_chea3_tsv(gene_set, output_filename):
    # The ChEA3 API URL.
    url = "https://maayanlab.cloud/chea3/api/enrich/"

    # Sends a JSON query to the ChEA3 API.
    # https://maayanlab.cloud/chea3/
    query_data = {
        "query_name":"myQuery",
        "gene_set":gene_set
    }
    result = requests.post(url, json=query_data)
    assert result.ok, f"Request failed with error code: {result.status_code}.\
    Check the ChEA3 website to make sure the request was correct."

    # ChEA3 produces several tables (objects) in its JSON response.
    # we get items from the table with this name:
    entry = "ENCODE--ChIP-seq"
    tfs = result.json()[entry]

    with open(output_filename, "w") as tsvfile:
        # Each item in the tfs is a dict. We make the table header using its keys.
        fieldnames = tfs[0].keys()

        writer = csv.DictWriter(tsvfile, fieldnames, delimiter="\t")
        writer.writeheader()
        for row in tfs:
            writer.writerow(row)

    print(f"Finished writing results to {output_filename}")

# maps (control, treatment) to dataframes
frames = {
    (MATRIGEL, AGRIN): oneA_0Msigs,
    (MATRIGEL, PAN_LAMANIN): twoP_0Msigs,
    (MATRIGEL, LAMANIN211): threeL_0Msigs,
    (MATRIGEL, COLLAGENI): fourC_0Msigs,
    (PLL, AGRIN): oneA_0Psigs,
    (PLL, PAN_LAMANIN): twoP_0Psigs,
    (PLL, LAMANIN211): threeL_0Psigs,
    (PLL, COLLAGENI): fourC_0Psigs
}

proteins = list(frames.keys()) + list(frames.values())
for protein in proteins:
    assert " " not in protein, f"Protein with name '{protein}' should not have a space"

for (pair, frame) in frames.items():
    control, treatment = pair

    print(f"Generating TSV for {control}, {treatment}...")
    gene_set = list(symbol for symbol in frame["Symbol"] if isinstance(symbol, str))

    make_chea3_tsv(gene_set, f"chea3_results/{control}_vs_{treatment}.tsv")

Finished writing results to chea3_output.csv


In [21]:
# Print the result's data (warning: long)
# result.json()

# Print the result's text (JSON if successful, HTML if failure)
# result.text

# Step 6: Graph of transcription factors
Generates graphs for each control-treatment pair using an R program written by Randall J. Eck.

In [29]:
!pwd

!Rscript ./transcriptionFactors.R sample_ENCODE_ChIP-seq.tsv sample_ENCODE_ChIP-seq.png

/home/ahdog/Projects/csci597k_final_project

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

sample_ENCODE_ChIP-seq.tsv sample_ENCODE_ChIP-seq.png[1] "Reading from sample_ENCODE_ChIP-seq.tsv printing to sample_ENCODE_ChIP-seq.png"
Rows: 118 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): Query Name, Set_name, TF, Library, Overlapping_Genes
dbl (7): Rank, Scaled Rank, Intersect, Set length, FET p-value, FDR, Odds Ratio

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
Removed 1 row containing missing values or values outside the scale range
(`geom_point()`). 
